## Supply Planning using Gurobi
This notebook demostrates how supply planning optmization can be done using Gurobi using a simplied Supply Planning Problem usecase.

Supply planning is managing the inventory produced by manufacturing to fulfil the requirements created from the demand plan.

### How many pallets do we need to ship to DC1 in the near future?

![suppy_planning_problem.png](img/suppy_planning_problem.png)

_source_: [Samir Saci](https://towardsdatascience.com/supply-planning-using-linear-programming-with-python-bff2401bf270)


### Problem Statement

As a Supply Planning manager of a mid-size manufacturing company, you received the feedback that the distribution costs are too high. Based on the analysis of the Transportation Manager this is mainly due to the stock allocation rules.

In some cases, your customers are not shipped by the closest distribution centre, which impacts your freight costs.

For simplicity, let's say we have the following points to take into consideration:

- Inbound Transportation Costs from the Plants to the Distribution Centers (DC) (\$/Carton).

- Outbound Transportation Costs from the DCs to the final customer (\$/Carton).

- Customer Demand (Carton).

This problem statement is based on [this](https://towardsdatascience.com/supply-planning-using-linear-programming-with-python-bff2401bf270) article. There the problem was solved using PuLP framework. 

### Loading data

- Loading the near future demand of stores (or customers) in terms of number of pallets required into a dataframe.

In [33]:
'''
Author: Dhruva Ahuja
'''
import pandas as pd

df_demand = pd.read_csv('data/df_demand.csv', index_col=0)
df_demand.set_index('STORE', inplace=True)
df_demand.head()

,DEMAND
STORE,
S1,244
S2,172
S3,124
S4,90
S5,158


### Loading data
- Inbound transportation cost from plant $P_i$ to distribution center $DC_j$ loaded into `df_inbound`.
- Outbound transportation cost from distribution center $DC_j$ to supplier $S_k$ loaded into `df_outbound`.

In [23]:
df_inbound = pd.read_csv('data/df_inbound_price.csv', index_col=0)
df_outbound = pd.read_csv('data/df_outbound_price.csv', index_col=0)
df_outbound = df_outbound.set_index('from')
df_outbound.head()

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,S191,S192,S193,S194,S195,S196,S197,S198,S199,S200
from,,,,,,,,,,,,,,,,,,,,,
D1,2.30,4.23,2.26,3.38,1.59,2.01,5.32,6.63,2.38,6.62,...,5.86,8.30,3.02,1.01,2.77,2.96,3.53,8.60,2.77,7.06
D2,5.31,2.18,8.52,8.34,4.59,1.04,1.89,6.45,8.35,3.32,...,7.54,2.11,4.33,1.54,4.75,7.84,8.21,4.51,3.27,3.13


In [3]:
df_inbound.head()

,D1,D2
P1,3.0,5.0
P2,2.3,6.6


In [4]:
# Number of plants, number of DCs, number of stores.
n_p, n_dc = df_inbound.shape
n_s = df_demand.shape[0]
n_p, n_dc, n_s

(2, 2, 200)

The optimization problem can be formulated as follows:

$$ TC = \sum_{i=1}^{2} \sum_{j=1}^{2} IB_{i, j} \times I_{i, j} + \sum_{j=1}^{2} \sum_{k=1}^{200} OB_{j, k} \times O_{j, k} $$

Where, 
- $IB_{i, j}$ is inbound cost ($/pallete) from plant $P_i$ to $DC_j$.
- $OB_{j, k}$ is outbound cost ($/pallete) from $DC_j$ to store $S_k$.


We have to minimize TC or Total Cost, subjected to some contraints (given later).

$$ \text{minimize} \quad {TC}$$

In [5]:
import gurobipy as gb
from gurobipy import GRB

m = gb.Model('supply_planning.lp')

# Define inbound and outbound variables, since the number of palletes (inbound or outbound) can only be an integer, 
# we specify the vtype to be integer. Hence making it a mixed integer linear programming problem.
I = m.addVars(n_p, n_dc, vtype=GRB.INTEGER, name='Inbound')
O = m.addVars(n_dc, n_s, vtype=GRB.INTEGER, name='Outbound')

# Inbound and Outbound cost per pallet is taken from their dataframes. Converted to numpy to utilize benefits of row-major indexing.
ic = df_inbound.to_numpy()
oc = df_outbound.to_numpy()
demand = df_demand.to_numpy().flatten()

m.update()

Restricted license - for non-production use only - expires 2025-11-24


In [6]:
objective = gb.quicksum(ic[i, j] * I[i, j] for i in range(n_p) for j in range(n_dc))
objective += gb.quicksum(oc[j, k] * O[j, k] for j in range(n_dc) for k in range(n_s))

m.setObjective(objective, GRB.MINIMIZE)
m.update()

Equality Constraint: Total inbound should be equal to total outbound for a distribution center.


$$ \sum_{i=0}^{2} I_{i, j} = \sum_{k=0}^{200} O_{j, k} ~~~~~~~~ \forall ~~ j \in \{1, 2\}$$

Inequality Contraint: Outbound from distribution centers should meet the customer demand.

$$ \sum_{j=0}^{2} O_{j, k} \ge D_k ~~~~~~~~ \forall ~~ k \in \{1, 2, \dotsc, 200\}$$

Integrality contraint: 
$$I_{i, j} \in \mathbb{Z}^+ ~~~ \forall (i, j), \\ O_{j, k} \in \mathbb{Z}^+ ~~~ \forall (j, k)$$

In [7]:
# Add the equality contraints
m.addConstrs((gb.quicksum(I[i, j] for i in range(n_p)) == gb.quicksum(O[j, k] for k in range(n_s))) for j in range(n_dc))

# Add the inquality contraints
m.addConstrs((gb.quicksum(O[j, k] for j in range(n_dc)) >= demand[k]) for k in range(n_s))

m.update()

In [8]:
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]


Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 202 rows, 404 columns and 804 nonzeros
Model fingerprint: 0xe2353170
Variable types: 0 continuous, 404 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Found heuristic solution: objective 290103.22000
Presolve removed 202 rows and 404 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 217189 290103 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.171893200000e+05, best bound 2.171893200000e+05, gap 0.0000%


In [27]:
import numpy as np

# Results of the computation by gurobi are stored in the variables that we defined earlier, which is a tupled dictionary,
# which can be formatted to show the results more clearly, e.g., # pallets that are shipped from P1 to DC2 can be easily seen in the resulting dataframe.
pd.DataFrame(np.array([x.X for x in I.values()]).reshape(-1, n_dc), columns=['DC1', 'DC2'], index=['P1', 'P2'], dtype=int)

,DC1,DC2
P1,0,6232
P2,25574,0


In [30]:
outbound_result = np.zeros(shape=(2, 200), dtype=int)
outbound_result

for j, k in O.keys():
    outbound_result[j, k] = O[j, k].X

demand_fulfilled = outbound_result.T
df_fulfilled = pd.DataFrame(demand_fulfilled, index=df_demand.index, columns=['DC1', 'DC2'])
df_fulfilled

,DC1,DC2
STORE,,
S1,244,0
S2,172,0
S3,124,0
S4,90,0
S5,158,0
...,...,...
S196,57,0
S197,52,0
S198,0,243


In [31]:
# basic evaluation to see if we were able to meet the demand or what was the spillage
# luckily, for our case, there was no extra pallets delivered than required.
(df_demand['DEMAND'] - df_fulfilled['DC1'] - df_fulfilled['DC2']).value_counts()

0    200
Name: count, dtype: int64